In [8]:
import requests
import os
import json
from pprint import pprint

# os.environ['BEARER_TOKEN'] = 'WRITE YOUR TOKEN HERE'
bearer_token = os.environ.get('BEARER_TOKEN')

def create_url(_ids):
#     tweet_fields = "tweet.fields=lang,author_id,attachments,context_annotations,conversation_id,created_at,entities,geo,id,public_metrics,withheld"
#     expansion = "expansions=author_id"
#     author_fields = "user.fields=id,name,username,created_at,description,entities,location,public_metrics,protected,verified,withheld"
    ids = "ids={}".format(_ids)
    tweet_fields = "tweet.fields=lang,created_at,geo,id,public_metrics,withheld"
    expansion = "expansions=author_id"
    user_fields = "user.fields=created_at,description,public_metrics,protected,verified,withheld"
    
    url = "https://api.twitter.com/2/tweets?{}&{}&{}&{}".format(ids, tweet_fields, expansion, user_fields)
    return url


def get_tweets(_ids):
    url = create_url(_ids)
    payload={}
    headers = {
      'Authorization': 'Bearer {}'.format(bearer_token),
      'Cookie': 'guest_id=v1%3A164999832374703747; guest_id_ads=v1%3A164999832374703747; guest_id_marketing=v1%3A164999832374703747; personalization_id="v1_+F68isE/iukb7yr8y66bOw=="'
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    return response.json()


In [2]:
import pandas as pd

def read_file(file_name):
    '''
    This function will read the file containing tweet ids sperating by lines (in groups)
    '''
    all_ids = pd.read_table(file_name,sep='\n',header=None)
    return all_ids

In [3]:
def separate_ids(ids):
    '''
    This function will split the sperate ids into a list
    '''
    seperated_ids = ids.split(",")
    return seperated_ids

In [141]:
def get_tweet_groups(groups_ids):
    '''
    This function will get tweets information using Twitter API in groups with indexing
    params groups_ids: all ids in lines (groups) retrieved from the orginal file 
    '''
    tweet_groups = {}
    no_access_first_tweets = []
    for group_index in range(len(groups_ids)):
        group_separated_ids = separate_ids(groups_ids[0][group_index])
        group_dict = {}
        
        # first check if the first tweet is valid to be accessed
        # if no access, only 'errors' will be returned
        first_tweet = get_tweets(group_separated_ids[0])
        if len(first_tweet) == 1:
            no_access_first_tweets.append(group_index)
            continue
        else:
            # check if the group contains more than 100 ids 
            # since multiple tweets lookup has 100 maxmimum
            if len(group_separated_ids) > 100:
                index = 0
                partial_group = {}

                # partition groups with more than 100 ids into maximum 100 each group
                for i in range(len(group_separated_ids)//100 + 1):
                    partial_group_ids = ','.join(group_separated_ids[index:index+100])
                    partial_group = get_tweets(partial_group_ids)
                    # initialise the dict with the information from the first partial group
                    if len(group_dict) == 0 and len(partial_group) != 1:
                        group_dict = partial_group
                    # combining partial groups into a whole group
                    else:
                        # there is any tweet valid to be accessed in the partial group
                        if len(partial_group) != 1:
                            group_dict['data'].extend(partial_group['data'])
                            group_dict['includes']['users'].extend(partial_group['includes']['users'])

                    index += 100

            else:
                full_group_ids = ','.join(group_separated_ids)
                group_dict = get_tweets(full_group_ids)

            tweet_groups[group_index] = group_dict
    return tweet_groups, no_access_first_tweets

In [157]:
def get_tweet_groups_start_end(groups_ids, start, end):
    '''
    This function will get tweets information using Twitter API in groups with indexing
    params groups_ids: all ids in lines (groups) retrieved from the orginal file 
    '''
    tweet_groups = {}
    no_access_first_tweets = []
    for group_index in range(start, end):
        group_separated_ids = separate_ids(groups_ids[0][group_index])
        group_dict = {}
        
        # first check if the first tweet is valid to be accessed
        # if no access, only 'errors' will be returned
        first_tweet = get_tweets(group_separated_ids[0])
        if len(first_tweet) == 1:
            no_access_first_tweets.append(group_index)
            continue
        else:
            # check if the group contains more than 100 ids 
            # since multiple tweets lookup has 100 maxmimum
            if len(group_separated_ids) > 100:
                index = 0
                partial_group = {}

                # partition groups with more than 100 ids into maximum 100 each group
                for i in range(len(group_separated_ids)//100 + 1):
                    partial_group_ids = ','.join(group_separated_ids[index:index+100])
                    partial_group = get_tweets(partial_group_ids)
                    # initialise the dict with the information from the first partial group
                    if len(group_dict) == 0 and len(partial_group) != 1:
                        group_dict = partial_group
                    # combining partial groups into a whole group
                    else:
                        # there is any tweet valid to be accessed in the partial group
                        if len(partial_group) != 1:
                            group_dict['data'].extend(partial_group['data'])
                            group_dict['includes']['users'].extend(partial_group['includes']['users'])

                    index += 100

            else:
                full_group_ids = ','.join(group_separated_ids)
                group_dict = get_tweets(full_group_ids)

            tweet_groups[group_index] = group_dict
    return tweet_groups, no_access_first_tweets

In [5]:
train_ids = read_file('../../project-data/train.data.txt')

In [133]:
# tweets_train = get_tweet_groups(train_ids)

In [161]:
tweets_train1, no_access_first_tweets1 = get_tweet_groups_start_end(train_ids, 0, 100)

In [160]:
pprint(tweets_train1)

{0: {'data': [{'author_id': '312362691',
               'created_at': '2020-04-15T00:28:03.000Z',
               'id': '1250219300389974016',
               'lang': 'en',
               'public_metrics': {'like_count': 0,
                                  'quote_count': 0,
                                  'reply_count': 1,
                                  'retweet_count': 0},
               'text': '5. Can regularly rinsing your nose with saline help '
                       'prevent infection with the new coronavirus? '
                       'https://t.co/ccMjhhD7BK'},
              {'author_id': '312362691',
               'created_at': '2020-04-15T00:27:20.000Z',
               'id': '1250219116993974272',
               'lang': 'en',
               'public_metrics': {'like_count': 0,
                                  'quote_count': 0,
                                  'reply_count': 1,
                                  'retweet_count': 0},
               'text': '4. Can eating g

                             'name': "One Man's Opinion",
                             'protected': False,
                             'public_metrics': {'followers_count': 243,
                                                'following_count': 851,
                                                'listed_count': 18,
                                                'tweet_count': 33503},
                             'username': '1MansOpinionz',
                             'verified': False},
                            {'created_at': '2008-11-27T19:52:42.000Z',
                             'description': 'This account is dedicated to '
                                            'politics with occasional forays '
                                            'into film/art/food/cats/etc. '
                                            '#Resistance NO marketers PLZ.',
                             'id': '17686959',
                             'name': 'THEE #VeryAsian politicalcat '
        

                 'title': 'Not Found Error',
                 'type': 'https://api.twitter.com/2/problems/resource-not-found',
                 'value': '1239562248990806016'}],
     'includes': {'users': [{'created_at': '2018-10-21T00:14:50.000Z',
                             'description': 'musician/writer/podcaster/youtuber/teacher/weirdo '
                                            '"getting the word out about '
                                            'kindness & fun" (-chris crofton) '
                                            '~he/any~ @breadsheetpod #zeitgang '
                                            '#defundelonmusk',
                             'id': '1053801749335683072',
                             'name': 'Hoss Bossman',
                             'protected': False,
                             'public_metrics': {'followers_count': 4913,
                                                'following_count': 5114,
                                                '

                             'id': '9609632',
                             'name': 'The Australian',
                             'protected': False,
                             'public_metrics': {'followers_count': 793390,
                                                'following_count': 558,
                                                'listed_count': 6088,
                                                'tweet_count': 262930},
                             'username': 'australian',
                             'verified': True},
                            {'created_at': '2011-11-19T06:42:00.000Z',
                             'description': 'Teacher. Dad. Grandpa. Author of '
                                            'Aussie Notables biography series. '
                                            'Founder of Green Barrow '
                                            'Publishing.',
                             'id': '416089358',
                             'name': 'Allan Drum

                                             'influence. @NoBrainsBeanie is my '
                                             'Trixie',
                              'id': '19733709',
                              'name': 'Speed',
                              'protected': False,
                              'public_metrics': {'followers_count': 882,
                                                 'following_count': 902,
                                                 'listed_count': 14,
                                                 'tweet_count': 64382},
                              'username': 'vwgtivw',
                              'verified': False},
                             {'created_at': '2012-12-26T00:55:30.000Z',
                              'description': 'Been poisoned by my constituents '
                                             '2021 World Series Champion 2021 '
                                             'National Champion #GoDawgs '
                     

...

                'text': '@jefft could have killed the hot pocket truck! Phew '
                        ':o “@CNN: Nestlé USA has issued a recall of Hot '
                        'Pockets. http://t.co/VYgWTgAQVK”'},
               {'author_id': '1412099658',
                'created_at': '2014-02-19T15:56:05.000Z',
                'id': '436167312006070272',
                'lang': 'en',
                'public_metrics': {'like_count': 0,
                                   'quote_count': 0,
                                   'reply_count': 0,
                                   'retweet_count': 0},
                'text': '@CNN Stop eating processed food America'},
               {'author_id': '282073527',
                'created_at': '2014-02-19T16:37:33.000Z',
                'id': '436177747128229888',
                'lang': 'en',
                'public_metrics': {'like_count': 3,
                                   'quote_count': 0,
                                   'reply_count':

                  'resource_type': 'tweet',
                  'section': 'data',
                  'title': 'Authorization Error',
                  'type': 'https://api.twitter.com/2/problems/not-authorized-for-resource',
                  'value': '436161668016795648'},
                 {'detail': 'Sorry, you are not authorized to see the Tweet '
                            'with ids: [436161733980217344].',
                  'parameter': 'ids',
                  'resource_id': '436161733980217344',
                  'resource_type': 'tweet',
                  'section': 'data',
                  'title': 'Authorization Error',
                  'type': 'https://api.twitter.com/2/problems/not-authorized-for-resource',
                  'value': '436161733980217344'},
                 {'detail': 'Sorry, you are not authorized to see the Tweet '
                            'with ids: [436161589621043200].',
                  'parameter': 'ids',
                  'resource_id': '4361615

                              'username': 'crabber21',
                              'verified': False},
                             {'created_at': '2012-01-10T23:33:41.000Z',
                              'description': 'future mrs dilf',
                              'id': '460627754',
                              'name': 'Emily',
                              'protected': False,
                              'public_metrics': {'followers_count': 175,
                                                 'following_count': 371,
                                                 'listed_count': 3,
                                                 'tweet_count': 8899},
                              'username': 'roseemoji',
                              'verified': False},
                             {'created_at': '2009-09-28T02:43:15.000Z',
                              'description': 'I love #Syracuse sports '
                                             '#CraftBeer #Jambands #Beer',
    

                'id': '499708805421486080',
                'lang': 'en',
                'public_metrics': {'like_count': 0,
                                   'quote_count': 0,
                                   'reply_count': 0,
                                   'retweet_count': 0},
                'text': '@nealcarter @ksdknews @MsBlackhart whether or not he '
                        'had a criminal record, what happened is still '
                        'completely disgusting and insane.'},
               {'author_id': '25717212',
                'created_at': '2014-08-15T01:49:22.000Z',
                'id': '500096881167724545',
                'lang': 'en',
                'public_metrics': {'like_count': 0,
                                   'quote_count': 0,
                                   'reply_count': 0,
                                   'retweet_count': 0},
                'text': '“@nealcarter: Confirmed: this is irrelevant. RT '
                        '@ksdknews:

                              'verified': False},
                             {'created_at': '2011-08-23T20:56:34.000Z',
                              'description': 'I have been married for 29 years '
                                             'and have one daughter of 25 My '
                                             'first love was music and I '
                                             'follow Liverpool fc',
                              'id': '360835699',
                              'name': 'Andrew Jackson',
                              'protected': False,
                              'public_metrics': {'followers_count': 33,
                                                 'following_count': 208,
                                                 'listed_count': 1,
                                                 'tweet_count': 242},
                              'username': 'ledzepplin72',
                              'verified': False},
                             

In [163]:
no_access_first_tweets1

[23, 25, 31, 43, 45, 47, 48, 50, 57, 67, 68, 71, 74, 76, 79, 95]

In [12]:
for group_index in range(len(train_ids)):
    group_ids =  train_ids[0][group_index]
    if group_ids.count(",") + 1 >= 100:
        print("line", group_index)
        print(group_ids.count(",") + 1)

line 74
193
line 93
171
line 106
165
line 180
143
line 243
101
line 267
101
line 279
124
line 280
111
line 310
237
line 360
124
line 365
183
line 373
312
line 391
202
line 406
200
line 425
200
line 499
226
line 531
198
line 601
152
line 625
202
line 678
256
line 683
217
line 711
325
line 714
177
line 744
185
line 769
178
line 784
107
line 1055
104
line 1079
125
line 1233
200
line 1369
129
line 1373
220
line 1379
156
line 1461
200
line 1466
190
line 1520
159
line 1609
119
line 1611
135
line 1629
156
line 1630
126
line 1708
140
line 1754
110
line 1804
200
line 1849
118
line 1851
214


In [ ]:
with open("data/train_data", "w",encoding='utf-8') as outfile:
        json.dump(tweets_train, outfile)

In [ ]:
with open("data/train_data",'r',encoding='utf-8') as f:
    train_data = json.load(f)

In [ ]:
pprint(train_data[0])